In [4]:
 
import pandas as pd
from pathlib import Path
import sqlite3

# Load the uploaded SQLite database
db_path = "../directus-quran/Quran_111.db"
conn = sqlite3.connect(db_path)

# Verify table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(table_query, conn)

# Debug: Print available tables
print("Available tables in the database:", tables["name"].tolist())

# Read the data only if the table exists
if "TheNobleQuran" in tables["name"].tolist():
    query = """
    SELECT sura_no, sura_name_en, sura_name_roman, ayah_no_surah, ayah_no_quran, ayah_translation_en
    FROM TheNobleQuran
    ORDER BY sura_no, CAST(ayah_no_surah AS INTEGER)
    """
    df = pd.read_sql_query(query, conn)
elif not tables.empty:
    conn.close()
    raise ValueError(f"Table 'TheNobleQuran' does not exist. Available tables: {tables['name'].tolist()}")
else:
    conn.close()
    raise ValueError("No tables exist in the database. Please check the database file.")

conn.close()

# Group by surah and create .mdx content
# Group by surah and create .mdx content
mdx_files = {}
for surah_no, group in df.groupby("sura_no"):
    surah_name_en = group["sura_name_en"].iloc[0]
    surah_name_roman = group["sura_name_roman"].iloc[0]
    
    # Sort the group by verse number to ensure correct order
    # group = group.sort_values('ayah_no_surah')
    
    # Create proper frontmatter with --- on separate lines
    content = "---\n"
    content += f"title: Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += f"description: Translation of Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += "---\n\n"
    
    # Add heading and content
    # content += f"# Surah {surah_no}: {surah_name_en} ({surah_name_roman})\n\n"
    for _, row in group.iterrows():
        content += f"**{row['ayah_no_surah']}.** {row['ayah_translation_en']}\n\n"
    
    # Pad surah number with leading zeros for correct sorting (e.g., 001, 002, ..., 114)
    filename = f"surah-{str(surah_no).zfill(3)}.mdx"
    mdx_files[filename] = content

# Save .mdx files to a downloadable zip
output_dir = Path("../astro-quran/src/content/docs/surahs")
output_dir.mkdir(exist_ok=True)

for filename, content in mdx_files.items():
    with open(output_dir / filename, "w", encoding="utf-8") as f:
        f.write(content)

# Zip the output directory
# import shutil
# zip_path = "./data/surah-mdx-files.zip"
# shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

# print(f"Generated {len(mdx_files)} MDX files with verses in correct numerical order")
# zip_path


Available tables in the database: ['sqlite_sequence', 'directus_migrations', 'directus_folders', 'directus_relations', 'directus_files', 'directus_fields', 'directus_operations', 'directus_notifications', 'directus_translations', 'directus_shares', 'directus_versions', 'directus_revisions', 'directus_users', 'directus_extensions', 'directus_sessions', 'directus_webhooks', 'directus_settings', 'directus_policies', 'directus_permissions', 'directus_access', 'directus_collections', 'directus_dashboards', 'directus_flows', 'directus_panels', 'directus_presets', 'directus_roles', 'directus_comments', 'directus_activity', 'TheNobleQuran']


##  quran_translations & quran_metadata Databases

In [17]:
 
import pandas as pd
from pathlib import Path
import sqlite3

# Load the uploaded SQLite database
db_path = "quran_translations.db"
conn = sqlite3.connect(db_path)

# Verify table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(table_query, conn)

# Debug: Print available tables
print("Available tables in the database:", tables["name"].tolist())

# Read the data only if the table exists
if "quran_metadata" in tables["name"].tolist():
    query = """
    SELECT surah_no, surah_name_en, surah_name_roman, ayah_no_surah, ayah_no_quran, ayah_ar, ayah_en, list_of_words
    FROM quran_metadata
    ORDER BY surah_no, CAST(ayah_no_surah AS INTEGER)
    """
    df = pd.read_sql_query(query, conn)
elif not tables.empty:
    conn.close()
    raise ValueError(f"Table 'quran_metadata' does not exist. Available tables: {tables['name'].tolist()}")
else:
    conn.close()
    raise ValueError("No tables exist in the database. Please check the database file.")

conn.close()

# Group by surah and create .mdx content
# Group by surah and create .mdx content
mdx_files = {}
for surah_no, group in df.groupby("surah_no"):
    surah_name_en = group["surah_name_en"].iloc[0]
    surah_name_roman = group["surah_name_roman"].iloc[0]
    
    # Sort the group by verse number to ensure correct order
    # group = group.sort_values('ayah_no_surah')
    
    # Create proper frontmatter with --- on separate lines
    content = "---\n"
    content += f"title: Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += f"description: Translation of Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += "---\n\n"
    content += 'import Aside from "~/components/Aside.astro"\n\n'
    content += 'import "~/styles/asides.css"\n\n'

    # Add heading and content
    # content += f"# Surah {surah_no}: {surah_name_en} ({surah_name_roman})\n\n"
    for _, row in group.iterrows():
        content += f"""
            <Aside type="quran" title="{row['ayah_no_surah']}">
            {row['ayah_ar']}
            </Aside>

            <Aside type="reference" title="{row['ayah_no_surah']}">
            {row['ayah_en']}
            </Aside>

        ---
        """

    # Pad surah number with leading zeros for correct sorting (e.g., 001, 002, ..., 114)
    filename = f"surah-{str(surah_no).zfill(3)}.mdx"
    mdx_files[filename] = content

# Save .mdx files to a downloadable zip
# output_dir = Path("../astro-quran/src/content/docs/surahs")
output_dir = Path("./data/mdx_surahs")
output_dir.mkdir(exist_ok=True)

for filename, content in mdx_files.items():
    with open(output_dir / filename, "w", encoding="utf-8") as f:
        f.write(content)

# Zip the output directory
# import shutil
# zip_path = "./data/surah-mdx-files.zip"
# shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

# print(f"Generated {len(mdx_files)} MDX files with verses in correct numerical order")
# zip_path


Available tables in the database: ['quran_translations', 'quran_metadata', 'sqlite_stat1', 'sqlite_stat4']


## Add ayah_no_quran

In [22]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('quran_translations1.db')
cursor = conn.cursor()

# Step 1: Add the new column (e.g., "seq_no")
cursor.execute("ALTER TABLE quran_translations ADD COLUMN ayah_no_quran INTEGER")

# Step 2: Fetch all rowids to assign sequence numbers
cursor.execute("SELECT rowid FROM quran_translations")
rows = cursor.fetchall()

# Step 3: Update each row with sequential numbers
for i, (rowid,) in enumerate(rows, start=1):
    cursor.execute("UPDATE quran_translations SET ayah_no_quran = ? WHERE rowid = ?", (i, rowid))

# Save changes
conn.commit()
conn.close()


## Translations in diff folders

In [ ]:
import pandas as pd
from pathlib import Path
import sqlite3

# Define paths to both databases
db_main_path = "quran_translations.db"
db_trans_path = "quran_translations1.db"

# Connect to both databases
conn_main = sqlite3.connect(db_main_path)
conn_trans = sqlite3.connect(db_trans_path)

# Load main quran_metadata
query_main = """
SELECT surah_no, surah_name_en, surah_name_roman, ayah_no_surah, ayah_no_quran, ayah_ar, ayah_en, list_of_words
FROM quran_metadata
ORDER BY surah_no, CAST(ayah_no_surah AS INTEGER)
"""
df_main = pd.read_sql_query(query_main, conn_main)

# Load translation table from second DB
query_trans = "SELECT * FROM quran_translations"
df_trans = pd.read_sql_query(query_trans, conn_trans)

# Close DB connections
conn_main.close()
conn_trans.close()

# Merge both datasets on ayah_no_quran
df_merged = pd.merge(df_main, df_trans, on="ayah_no_quran", how="left")

# Get all translation language columns
translation_columns = [
    "bangla_translation", "chinese_translation", "german_translation", "italian-piccardo_translation",
    "japanese_translation", "malayalam_translation", "norwegian_translation", "persian_translation",
    "portuguese_translation", "russian_translation", "turkish_translation", "urdu_translation",
    "uzbek_translation", "english_translation"
]

# Base output directory
base_output_dir = Path("./data/mdx_translations")
base_output_dir.mkdir(exist_ok=True)

# Loop over each translation language
for lang in translation_columns:
    mdx_files = {}
    for surah_no, group in df_merged.groupby("surah_no"):
        surah_name_en = group["surah_name_en"].iloc[0]
        surah_name_roman = group["surah_name_roman"].iloc[0]

        content = "---\n"
        content += f'title: Surah {surah_no} - {surah_name_en} ({surah_name_roman})  \n'
        content += f'description: Translation of Surah {surah_no} - {surah_name_en} ({surah_name_roman}) in {lang.replace("_translation", "").capitalize()}\n'
        content += "---\n\n"
        content += 'import Aside from "~/components/Aside.astro"\n\n'
        content += 'import Spacer from "~/components/Spacer.astro"\n\n'
        content += 'import "~/styles/asides.css"\n\n'
        content += 'import VerseCard from "~/components/VerseCard.astro"\n\n'

        for _, row in group.iterrows():
            translation = row.get(lang, "").strip()
            if translation:
                content += f"""
    <VerseCard 
    verseNumber={{{row['ayah_no_surah']}}}
    chapterNumber={{{row['surah_no']}}}
    arabicText="{row['ayah_ar']}"
    translation="{translation}"
    transliteration="{row.get('transliteration', '')}"
    />
    
    <Spacer size="0.5rem" />
                """

        filename = f"surah-{str(surah_no).zfill(3)}.mdx"
        mdx_files[filename] = content

    # Write to folder per language
    lang_dir = base_output_dir / lang.replace("_translation", "") / "surahs"
    lang_dir.mkdir(parents=True, exist_ok=True)

    for filename, content in mdx_files.items():
        with open(lang_dir / filename, "w", encoding="utf-8") as f:
            f.write(content)

 


COPY ENGLISH TRANSLATION FROM METADTA TO QURAN TRANSLATIONS TABLE

In [29]:
import sqlite3

# Paths to the source and target databases
db_main_path = "quran_translations.db"
db_trans_path = "quran_translations1.db"

# Connect to the source DB
src_conn = sqlite3.connect(db_main_path)
src_cursor = src_conn.cursor()

# Connect to the destination DB
dest_conn = sqlite3.connect(db_trans_path)
dest_cursor = dest_conn.cursor()

# Step 1: Read ayan_en from source DB
src_cursor.execute("SELECT rowid, ayah_en FROM quran_metadata")
data = src_cursor.fetchall()

# Step 2: Add new column to destination table if it doesn't exist
try:
    dest_cursor.execute("ALTER TABLE quran_translations ADD COLUMN english_translation TEXT")
except sqlite3.OperationalError as e:
    if "duplicate column name" in str(e):
        print("Column 'english_translation' already exists, continuing...")
    else:
        raise

# Step 3: Update destination table row by row
for rowid, english_text in data:
    dest_cursor.execute(
        "UPDATE quran_translations SET english_translation = ? WHERE rowid = ?",
        (english_text, rowid)
    )

# Commit and close
dest_conn.commit()
src_conn.close()
dest_conn.close()

print("✅ Column copied successfully.")


✅ Column copied successfully.
